# AML Framework Code

In [ ]:
# Import azureml sdk packages
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.estimator import Estimator
import azureml.core

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# Set variables
cluster_name     = "larger-cluster"
experiment_name  = "fashion-mnist"
environment_file = "fashion-env.yml"
environment_name = "fashionenv"
training_script = 'fashion-mnist-code.py'

In [ ]:
# Log into a workspace
try:
    ws = Workspace.from_config()
    print("Using workspace:",ws.name,"in region", ws.location)
except:
    print('Workspace not found')

In [ ]:
# Choose computer cluster
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Using compute cluster:', cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=0,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

In [ ]:
# Connect to the default datastore
ds = ws.get_default_datastore()

In [ ]:
# Create an experiment or connect if it exists
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# create an environment from conda yml file
fashionenv = Environment.from_conda_specification(name = environment_name ,
                                          file_path = environment_file )
print(fashionenv)

fashionenv.register(workspace=ws)

In [ ]:
# List environments
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("fashion"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
# Create training folder and move train.py file into it
import os
import shutil

script_folder = os.path.join(os.getcwd(), "fashion-mnist")
os.makedirs(script_folder, exist_ok=True)
shutil.copy(training_script, script_folder)

In [ ]:
# create ScriptRunConfig object and connect environment
from azureml.core import ScriptRunConfig

args = ['--data-folder', '--regularization', 0.5]

src = ScriptRunConfig(source_directory=script_folder,
                      script=training_script, 
                      arguments=args,
                      compute_target=cluster,
                      environment=fashionenv)

In [ ]:
# Submit a run
run = exp.submit(src)
aml_url = run.get_portal_url()
print(aml_url)

In [ ]:
# View Jupyter widget
from azureml.widgets import RunDetails
RunDetails(run).show()